# 02. Evaluate with predefined scorer

In [ ]:
%run ./00_setup

## Test inference

In [ ]:
sample_lease_text = """
Exhibit 10.33
Big Tree Office Lease Contract
(Translation)
Lessor: Shantou Youbang International Express Supervise Center Co., Ltd, (“Party A”)
Lessee: Big Tree International Co., Ltd. (“Party B”)
Article 1. Property location
The property is located at Waisha Shanfen Road, Taixinglong Yard, Building 7,8 and 9, Longhu District,The office has 9746.65 square meters.
Articles 2. Lease Term
The lease contract shall be effective from October 1, 2012 and ending on December 31, 2017. The first three month shall be deemed as decoration period, the rent for the period shall be used as deposit and return to Party B after the expiration of this lease agreement.
Articles 3. Rent and Payment
The rent per month is RMB 120,000, the payment shall be paid quarterly. If the rent is paid by any foreign currency, the ratio shall be the mid price listed by People’s Bank of China on that certain day.
Article 4. Property Maintenance
1. The lessor is responsible for the regulated property maintenance.
2. With Party A’s consent, Party B’s regulated maintenance shall be paid by Party B’s first. Party A shall reduce the certain expenses from the rent payment according to the invoice provided by Party B.
3. Any repairmen cost more than RMB 5,000 shall be paid by Party A, otherwise, shall be paid by Party B.
4. With Party A’s consent, Party B has the right to remodel the property. But the remodeling shall not affect the property structure.
Article 5.
1. If Party A transfers the property to a third party, this agreement shall bind to the new owner.
2. Party A shall notice Party B three months prior to the sale of the property; Party B has the priority to purchase the property at the same term.
3. Party B shall acquire Party A’s consent before subleasing the property.
Article 6. Rights and Obligations of Both Parties
1. Party A shall provide the office as listed, otherwise, Party A shall pay 20% of the rent to Party B.
2. If Party B fails to pay the rent on due date, Party B shall pay 5/10000 of the rent each day delayed as the penalty.
3. Party B has the right to reject other payments other than rent.
4. If Party B sublease the office to a third party without Party A’s consent, Party A has the right to terminate the lease agreement.
5. After the expiration of this agreement, if Party B continue to use the office, Party B shall pay 5/10000 of each day of using
Article 7. Dissolution of the Lease
1. Neither party shall be responsible for the loss caused by force majeure.
2. Neither party shall be responsible for the loss caused by government reconstruction decision.
Article 8. Laws and Dispute Resolution
All disputes between the Lessor and Lessee shall be settled by negotiation, if cannot be resolved through negotiation, either party can bring it to the arbitration committee of industry and commerce for an arbitration.
Article 9. Other Regulations (None)
Article 10. This contract can be amended after the negotiation between both parties.
Lessor: Seal
Lessee: Seal
Date: September 30, 2012
Signing Place: Shantou City
"""

# test run
extract_lease_data(query=sample_lease_text)

## Load evaluation data as records

In [ ]:
import pandas as pd

eval_dataset = mlflow.genai.datasets.get_dataset(
    uc_table_name=f"{CATALOG}.{SCHEMA}.{EVAL_TABLE}",
)

eval_records = eval_dataset.to_df()[["inputs", "expectations"]].to_dict(
    orient="records"
)

## Run eval with predefined scorers

In [ ]:
# here's a list of predefined scorers
from mlflow.genai.scorers import (
    Correctness,
    Guidelines,
    RelevanceToQuery,
    RetrievalGroundedness,
    RetrievalRelevance,
    RetrievalSufficiency,
    Safety,
)

In [ ]:
# Run predefined scorers that require ground truth
with mlflow.start_run(run_name="Eval with predefined scorers"):
    mlflow.genai.evaluate(
        data=eval_records,
        predict_fn=extract_lease_data,
        scorers=[
            Correctness(),  # since we have labels
            RelevanceToQuery(),
            Safety(),
        ],
    )